In [7]:
import requests
from bs4 import BeautifulSoup as bs
import pandas as pd
from tqdm import tqdm

### API 연결 및 데이터 파씽

In [19]:
def url_parser(sDate, eDate, row=10):
    ### API 연결 및 데이터 파씽
    key="VFYXI%2F3A5kIk7QVsO1epAlZSb6ZtCO9N5VC%2FMF3pW6oZ7%2BsFrvH1Y9y0UfmVzTnA0k2FasoNM6t6JsY7%2BEwNEw%3D%3D"

    # 기본 url + 사용자 key
    url="http://api.visitkorea.or.kr/openapi/service/rest/KorService/searchFestival?serviceKey=" + key
    url=url+"&MobileOS=ETC&MobileApp=AppTest&arrange=A&listYN=Y"   # 기본 필수 옵션
    url=url+"&numOfRows="+ str(row)   # 한페이지 출력 데이터 개수
    url=url+"&eventStartDate="+str(sDate)  # 조회 시작일
    url=url+"&eventEndDate="+str(eDate)
    #print(url+"\n\n")
    xml_soup=requests.get(url)
    if xml_soup.status_code != 200:
        print("API를 연결하지 못하였습니다.")
        exit()
        
    soup=bs(xml_soup.content, 'html.parser')  # xml_soup.text 변경 가능
    
    return soup


### 파씽한 데이터에서 원하는 데이터 추출 및 DataFrame로 변경

In [21]:
def tour_info(sDate, eDate, soup):
    ### 파씽한 데이터에서 원하는 데이터 추출 및 DataFrame로 변경
    row=soup.find('totalcount').text
    re_soup=url_parser(sDate, eDate, row)
    
    items=re_soup.find_all('item')
    df_list=[]
    
    for item in tqdm(items, desc="진행:"):
        staD=item.find('eventstartdate').text      # 축제 시작일
        endD=item.find('eventenddate').get_text()  # 축제 종료일
        titD=item.find('title').get_text()         # 축제명
        try:
            addD=item.find('addr1').get_text()         # 주소
        except:
            addD="온라인개최"
        try:
            telD=item.find('tel').get_text()           # 전화
        except:
            telD="-"
        try:
            lngD=item.find('mapx').text                # 경도
            latD=item.find('mapy').get_text()          # 위도
        except:
            lngD=0
            latD=0
        
        df_list.append({"시작일":staD, "종료일":endD, "축제명":titD,
                        "주소":addD, "연락처":telD, '위도':latD, '경도':lngD})
        
    df1=pd.DataFrame(df_list)
    
    return df1

### 코드 실행

In [23]:
# if __name__=="__main__":
sDate=input("조회 시작일:(예:20200101):")
eDate=input("조회 종료일:(예:20201231):")

soup=url_parser(sDate, eDate)
df1=tour_info(sDate, eDate, soup)
df1.to_csv('c:/pydata/tour_'+str(sDate)+"_"+str(eDate)+".csv", index=False)

display(df1) #print(df1)

조회 시작일:(예:20200101):20190101
조회 종료일:(예:20201231):20211231


진행:: 100%|███████████████████████████████████████████████████████████████████████| 727/727 [00:00<00:00, 5564.47it/s]


,시작일,종료일,축제명,주소,연락처,위도,경도
0,20210306,20211030,가무악극으로 만나는 토요 상설공연,충청남도 부여군 규암면 백제문로 388,041-832-5765,36.3048428766,126.8997016069
1,20210320,20210630,가산수피아 '봄의 정원',경상북도 칠곡군 가산면 학하들안2길 105,054-971-9861,36.0861545517,128.4850691574
2,20210116,20211231,가족끼리 빙어체험 2021,경기도 양평군 단월면 곱다니길 55-2 수미마을 방문객센터,031-775-5205,37.5145504080,127.6574135888
3,20210301,20210606,가족끼리딸기 캠피닉,경기도 양평군 단월면 곱다니길 55-2,031-775-5205,37.5144948588,127.6573663730
4,20210225,20210228,가족오페라 <마술피리> 2021,대구광역시 북구 호암로 15,053-666-6170,35.8835072863,128.5922570230
...,...,...,...,...,...,...,...
722,20211013,20211017,[문화관광축제] 추억의 충장축제 2021,광주광역시 동구 서남로 1,062-608-2421~4,35.1454973445,126.9231294053
723,20210523,20211130,[문화관광축제] 춘천마임축제,강원도 춘천시 전역,033-242-0585,37.8766011603,127.7178558012
724,20210812,20210821,[문화관광축제] 통영한산대첩축제,경상남도 통영시 통영해안로 328,055-644-5222,34.8438295639,128.4241721001
725,20210611,20210613,[문화관광축제] 한산모시문화제,충청남도 서천군 한산면 충절로 1089,041-950-4256,36.0799251950,126.7987054614
